Explore the graph_tool package, in order to do a compare & contrast of SBMs and Infomap

In [65]:
import graph_tool
import graph_tool.inference
import numpy as np
import matplotlib.pyplot as plt
from palettable.colorbrewer.qualitative import Set3_12
%matplotlib inline

In [91]:
edgelistArg= "3p8t.4.0.dat"
g = graph_tool.load_graph_from_csv(edgelistArg, directed=False,
                                   skip_first=True, csv_options={"delimiter": " "})
print(g)

In [92]:
state = graph_tool.inference.minimize_nested_blockmodel_dl(g, deg_corr=True)

In [93]:
hierarchical_partition = []
for i, innerstate in enumerate(state.get_levels()):
    hierarchical_partition.append(list(state.project_level(i).b.get_array()))
print(hierarchical_partition)

In [94]:
state.draw()

In [68]:
def plotStripeDiagram(data):
    """
    Plot the partition as a set of "stripe" plots.
    """
    # Convert the list (or nested list) to a numpy array, for use with imshow.
    stripes = np.asarray(data, dtype=int)
    numPlots =  np.shape(stripes)[0]

    fig, axes = plt.subplots(nrows=numPlots, figsize=(5, 5), sharex=True)

    for i, ax in enumerate(axes):
        ax.imshow(
            np.vstack(
                2 * [stripes[i, :]]),  # vstack otherwise imshow complains
            aspect=100,
            cmap=Set3_12.mpl_colormap,
            interpolation="nearest")
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_visible(False)

#     plt.suptitle(self.pdbref)
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.xlabel('Residue number')
    plt.show()

In [69]:
plotStripeDiagram(hierarchical_partition)

The above shows the block model hierarchy. Need to reverse the order and trim the "all-one-community" level.

In [95]:
hierarchical_partition =hierarchical_partition[::-1][1:]

In [73]:
plotStripeDiagram(hierarchical_partition)

In [76]:
print(len(hierarchical_partition[-1]))
print(len(set(hierarchical_partition[-1])))

In [78]:
state = state.copy(sampling=True)

In [79]:
graph_tool.inference.mcmc_equilibrate(state)

In [80]:
state.draw()

In [81]:
hierarchical_partition_new = []
for i, innerstate in enumerate(state.get_levels()):
    hierarchical_partition_new.append(list(state.project_level(i).b.get_array()))
hierarchical_partition_new = hierarchical_partition_new[::-1][1:]

In [82]:
plotStripeDiagram(hierarchical_partition_new)

So it's looking like mcmc_equilibriate, as I'm currently doing it, isn't the one. 

Compare with Infomap:

In [97]:
%load_ext autoreload
%autoreload 2

import proteinnetworks

In [84]:
db = proteinnetworks.database.Database()

In [111]:
inputArgs = {"scaling": 4.5,
"edgelisttype": "residue",
"hydrogenstatus": "noH",
"pdbref": "3p8t",
"database": db}
network = proteinnetworks.network.Network(**inputArgs)
from bson.objectid import ObjectId
partitionArgs = {"pdbref": "3p8t",
                 "edgelistid": ObjectId(network.edgelistid),
                 "detectionmethod": "Infomap",
                "N": 100,
                "database": db}
infomap_partition = proteinnetworks.partition.Partition(**partitionArgs)
infomap_partition.plotStripeDiagram(includePFAMDomains=True)

In [96]:
plotStripeDiagram(hierarchical_partition)

In [110]:
proteinnetworks.insight.getNMI(
    [x+1 for x in hierarchical_partition[0]], infomap_partition.data[0])